## Cyber Security Selector Group Chat

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

print(os.getenv('CHAT_MODEL'))

In [ ]:
import os
import logging
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient


logger = logging.getLogger("autogen_core") 
logger.setLevel(logging.WARNING)


project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(), conn_str=os.environ["AIPROJECT_CONNECTION_STRING"]
)

base_url = project_client.inference.get_azure_openai_client(
    api_version="2024-06-01").base_url

api_endpoint = f'https://{base_url.host}/'

api_key = project_client.inference.get_azure_openai_client(
    api_version="2024-06-01").api_key

deployment_name = os.environ["CHAT_MODEL"]

aoai_client = AzureOpenAIChatCompletionClient(
    azure_endpoint=api_endpoint,
    model="gpt-4o-mini",
    azure_deployment=deployment_name,
    api_key=api_key,
    api_version="2024-06-01"
)

In [ ]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.messages import TextMessage
from autogen_agentchat.ui import Console
from autogen_core import CancellationToken


async def assistant_run_stream(agent: AssistantAgent, content: str) -> None:
    # Option 1: read each message from the stream.
    # async for message in agent.on_messages_stream(
    #     [TextMessage(content="Find information on AutoGen", source="user")],
    #     cancellation_token=CancellationToken(),
    # ):
    #     print(message)

    # Option 2: use Console to print all messages as they appear.
    await Console(
        agent.on_messages_stream(
            [TextMessage(content=content, source="user")],
            cancellation_token=CancellationToken(),
        )
    )

### Agent 1: Time keeper

In [ ]:
from autogen_agentchat.agents import AssistantAgent
from user_functions import fetch_current_datetime


time_keeper = AssistantAgent(
    "time_keeper",
    description="An AI agent specialized in providing accurate and real-time time information.",
    tools=[fetch_current_datetime],
    model_client=aoai_client,
    system_message="""
    # Role
    You are TimeKeeper, a specialized AI agent designed to provide accurate and real-time time information. Your primary responsibility is to serve as a reliable and precise source for current time data, ensuring users and systems can access the exact time whenever needed. You are adaptable and capable of formatting the time output to suit various user needs or technical requirements.

    # Tasks
    1. **Provide Current Time**:
        - Always return the current time based on the user's specified timezone or system default timezone.
        - Ensure the time is accurate to the second.

    2. **Format Time Output**:
        - Support multiple time formats based on user requests, such as:
            - 12-hour format (e.g., "2:30 PM").
            - 24-hour format (e.g., "14:30").
            - ISO 8601 format (e.g., "2023-10-12T14:30:00Z").
            - Unix timestamp (e.g., "1697111400").
        - If no format is specified, default to a standard 24-hour format with the date (e.g., "2023-10-12 14:30:00").

    3. **Timezone Customization**:
        - Provide time information for any specified timezone (e.g., "UTC", "EST", "PST").
        - If no timezone is specified, default to the system's local timezone.

    4. **Error Handling**:
        - Handle invalid timezones, formats, or calculation requests gracefully by providing clear error messages or alternative suggestions.

    # Goal
    Your ultimate goal is to provide **accurate, reliable, and user-friendly time information** for any request. Ensure clarity, precision, and adaptability in your responses to meet the varying needs of users and systems.
    """,
)

In [ ]:
# Use asyncio.run(assistant_run_stream()) when running in a script.
user_message = "Hello, what time is it now?"
await assistant_run_stream(time_keeper, user_message)

### Agent 2: Cyber collector

In [ ]:
from autogen_agentchat.agents import AssistantAgent
from user_functions import fetch_knowledge


cyber_collector = AssistantAgent(
    "cyber_collector",
    description="The cyber collector can collect information about cyber security from its knowledge base.",
    tools=[fetch_knowledge],
    model_client=aoai_client,
    system_message="Hello, you are helpful assistant and can search information related to cybersecurity."
)

In [ ]:
# Use asyncio.run(assistant_run_stream()) when running in a script.
user_message = "Collect the latest cybersecurity threat information as well as their current metrics."
await assistant_run_stream(cyber_collector, user_message)

### Agent 3: DB Reader

In [ ]:
from autogen_agentchat.agents import AssistantAgent
from user_functions import fetch_data


db_reader = AssistantAgent(
    "db_reader",
    description="This agent fetches data from a specified SQLite database table, saves it as a CSV file, and returns the status and file path in JSON format.",
    tools=[fetch_data],
    model_client=aoai_client,
    system_message="Hello, you are helpful assistant.",
)

In [ ]:
# Use asyncio.run(assistant_run_stream()) when running in a script.
# user_message = "Hello, get the intrution attemps data."
user_message = "Hello, get the incident detection rate data."
await assistant_run_stream(db_reader, user_message)

### Agent 4: Data Analyzer

In [ ]:
from autogen_agentchat.agents import AssistantAgent
from user_functions import analyze_data


data_analyzer = AssistantAgent(
    "data_analyzer",
    description="An agent that analyzes data using forecasting or anomaly detection.",
    tools=[analyze_data],
    model_client=aoai_client,
    system_message="Hello, you are helpful assistant.",
)

In [ ]:
# run the agent
async def run_data_analysis_stream() -> None:

    await Console(
        data_analyzer.on_messages_stream(
            [
                TextMessage(content="""The intrusion attempts data has been successfully fetched and saved as a CSV file. You can find it at the following path: ./data/intrusion_attempts_20250209_221624.csv.
    
    The incident detection data has been successfully fetched and saved as a CSV file. You can find it at the following path: ./data/incident_detection_rate.csv
    """, source="assistant"),
                TextMessage(
                    content="Hello, create a 10-day forecast using the intrusion attempts data. And detect anomalies in the incident detection data.", source="user")
            ],
            cancellation_token=CancellationToken(),
        )
    )

# Use asyncio.run(assistant_run_stream()) when running in a script.
await run_data_analysis_stream()

### Agent 5: Security Evaluator

In [ ]:
from autogen_agentchat.agents import AssistantAgent
from user_functions import generate_radar_chart


security_evaluator = AssistantAgent(
    "security_evaluator",
    description="An agent that assists users in generating radar charts for cybersecurity metrics.",
    tools=[generate_radar_chart],
    model_client=aoai_client,
    system_message="""
# Role
You are "Security Evaluator," an AI agent designed to assess and visualize cybersecurity metrics using radar charts. Your primary goal is to assist users in generating a radar chart that visually represents multiple security-related categories and their corresponding values. Use the provided tool to create these visualizations and guide the user effectively in providing the necessary input.

# Tasks
1. Collecting a list of categories (e.g., "Vulnerability Score," "Detection Rate") and their corresponding values (e.g., scores on a uniform scale like 0–10) from the user.
2. Ensuring the input is valid (e.g., the number of categories matches the number of values, and values are within a valid range).
3. Using the generate_radar_chart function to generate a radar chart based on the user's input.
4. Explaining the result to the user:
    - If successful, provide the file path to the saved radar chart.
    - If an error occurs, describe the issue and guide the user to fix it.

# Goal
You are polite, concise, and clear in your communication. Provide helpful examples when the user is unsure about what input to provide. Always ensure that the user understands the requirements for generating the radar chart.
"""
)

In [ ]:
# Use asyncio.run(assistant_run_stream()) when running in a script.
user_message = """I’m analyzing my organization’s cybersecurity posture and need a radar chart to visualize it. Here’s how the metrics look:

The Vulnerability Score is 8.
The Detection Rate is 7.
The Response Time is rated at 6.
Our Threat Intelligence capability scores a 9.
Finally, the System Uptime is at an 8.
Could you help me create a radar chart for these?"""
await assistant_run_stream(security_evaluator, user_message)

### Agent 6: Report Writer

In [ ]:
from autogen_agentchat.agents import AssistantAgent
from user_functions import generate_report


report_writer = AssistantAgent(
    "report_writer",
    description="An advanced AI agent responsible for integrating information and analysis results from various agents to produce a professional, detailed, and actionable cybersecurity report.",
    tools=[generate_report],
    model_client=aoai_client,
    system_message="""
# Role
You are Report Writer, an advanced AI agent in a multi-agent cybersecurity system. Your primary responsibility is to integrate information and analysis results from various agents to produce a professional, detailed, and actionable cybersecurity report. The report serves as the final deliverable in the system and is used to inform stakeholders about the current cybersecurity landscape, potential threats, and recommended defensive strategies. Your output must adhere to a structured format, ensuring clarity, professionalism, and compliance with industry standards.

# Tasks
1. **Information Integration**:
    - Gather and synthesize structured data from the following sources:
        - **Cyber Collector**: Provide insights on threat intelligence, defensive strategies, and industry standards.
        - **DB Reader**: Access real-time data from the database for inclusion in the report.
        - **Data Analyzer**: Integrate results from anomaly detection, trend analysis, and predictive models.
    - Organize and categorize the information based on the specific needs of the report to ensure a logical flow.

2. **Report Generation**:
    - Create a **comprehensive cybersecurity report** following a professional and structured format. The report must include:
        - **Executive Summary**: A concise overview of key findings and actionable recommendations.
        - **Detailed Analysis**: In-depth examination of the data, enhanced with visualizations (e.g., charts, graphs).
        - **Recommendations**: Actionable steps to mitigate risks and improve cybersecurity posture.
        - **Appendices**: Supplementary data, charts, and references for further context.
    - Convert the finalized report into a visually appealing PDF document with appropriate formatting, font styles, and a professional layout.

# Report Structure
Your generated report must adhere to the following structure:

1. **Title Page**:
    - Report Title: "Cybersecurity Threat and Defense Report"
    - Date: [Current Date]
    - Optional: Include the Client or Organization Name if provided.

2. **Table of Contents**:
    - If supported, create a clickable/interactive table of contents (e.g., in PDF format with bookmarks).

3. **Main Sections**:
    - **Overview**: Summarize the current cybersecurity landscape and the purpose of the report.
    - **Threat Intelligence**: Provide a detailed overview of recent threats, their potential impacts, and associated risks.
    - **Data Analysis Results**: Present findings from anomaly detection, trends, and forecasts, supported by visual elements (e.g., bar charts, line graphs, radar plots).
    - **Defensive Recommendations**: Outline actionable strategies to mitigate identified risks and enhance the organization’s cybersecurity posture.
    - **Compliance Assessment**: Evaluate compliance with industry standards, regulations, or best practices.
    - **Conclusion**: Recap key takeaways, major findings, and recommended next steps.

4. **Appendices**:
    - Include additional datasets, technical details, or supporting visualizations as required.

# Features of the PDF Report
1. **Professional Design**:
    - Use visually appealing layouts, consistent font styles, and clear section headings.
    - Ensure the report is easy to navigate with a clean and structured design.
2. **Interactive Elements**:
    - If applicable, include clickable links in the Table of Contents for easier navigation.
3. **Visual Enhancements**:
    - Integrate relevant charts, graphs, and images to make the data more digestible and engaging.
4. **Error-Free Content**:
    - Ensure the final PDF is free of typos, formatting errors, or inconsistencies.

# Goal
Your ultimate goal is to produce a **polished, actionable, and visually compelling cybersecurity report** in PDF format that meets the user’s needs. The report must reflect a deep understanding of the data provided by other agents, offering meaningful insights and strategies to enhance cybersecurity posture. Prioritize clarity, professionalism, and user satisfaction in every aspect of the report creation process.
"""
)

In [ ]:
# run the agent
async def write_report_stream() -> None:

    await Console(
        report_writer.on_messages_stream(
            [
                TextMessage(
                    content="Generate a cybersecurity report for Contoso.", source="user"),
                TextMessage(
                    content="Hello, the current date and time is January 3, 2025, 05:53:07 UTC.", source="assistant"),
                TextMessage(
                    content="""Cybersecurity KPIs are essential for measuring the effectiveness of an organization’s security posture. Key metrics include Intrusion Attempts (unauthorized access attempts), Malicious Traffic Proportion (percentage of harmful network activity), Incident Detection Rate (percentage of threats identified), MTTD/MTTR (time to detect/respond to incidents), and Data Breach Volume (amount of compromised data). Other critical KPIs include Patch Compliance (rate of systems updated), False Positive/Negative Rates (accuracy of detection tools), User Awareness Metrics (training effectiveness, e.g., phishing test results), and Vulnerability Management (percentage of critical vulnerabilities remediated). Additional metrics cover Privileged Access Management, Endpoint Protection, Regulatory Compliance, and Cost of Incidents. Tracking these KPIs helps organizations mitigate risks, improve defenses, and ensure compliance.
                    Our current metrics include Intrusion Attempts and Incident Detection Rate.""", source="assistant"),
                TextMessage(
                    content="""I have retrieved the requested data and saved it to the specified file paths. Here's the result:
                    1. Data for Intrusion Attempts has been saved to: ./data/Intrusion Attempts.csv
                    2. Data for Incident Detection Rate has been saved to: ./data/Incident Detection Rate.csv
                    If you need further assistance in processing, analyzing, or visualizing this data, let me know!""", source="assistant"),
                TextMessage(
                    content="""After analyzing the data from the provided CSV files, I have generated and reviewed the visualizations for Intrusion Attempts and Incident Detection Rate. Below are the detailed insights and recommendations based on the findings:

                    # 1. Intrusion Attempts: Forecasted vs Actual Values
                        ![Intrusion Attempts Forecast](./figures/Intrusion_Attempts_forecast_plot.png)

                        - Trend Analysis: The data reveals a clear upward trend in intrusion attempts over time, indicating a steady increase in malicious activities targeting the system.
                        - Seasonality: There is evidence of recurring patterns, with peaks and troughs occurring at regular intervals. This cyclical behavior suggests periodic threats or exploitation of recurring vulnerabilities.
                        - Forecast Accuracy: The forecasted values (pink line) closely align with the actual values (blue line), demonstrating the reliability of the forecasting model used. The forecast projects a continued rise in intrusion attempts.
                    ## Recommendations:

                        1. Investigate the root causes of periodic spikes in intrusion attempts and address recurring vulnerabilities.
                        2. Enhance perimeter defenses, such as firewalls and intrusion prevention systems (IPS), to mitigate the projected increase in threats.
                        3. Use the forecast data to proactively allocate resources and improve incident response readiness.

                    # 2. Incident Detection Rate: Anomalies on Actual Values
                        ![Intrusion Attempts Forecast](./figures/Incident_Detection_Rate_anomalies_plot.png)

                        - Anomalies Identified: The model detected multiple anomalies (red circles), representing instances where the detection rate significantly deviated from expected behavior. These could indicate system misconfigurations, unusual attack patterns, or delayed responses.
                        - Overall Stability: The detection rate generally remains within the expected range (purple shaded region), indicating consistent performance of the detection mechanisms. However, periodic spikes and dips highlight areas for further investigation.
                        - Spike Analysis: Spikes in detection rates may correlate with periods of heightened activity or improved detection capabilities, while dips could reflect potential monitoring gaps or delayed responses.
                    ## Recommendations:

                        1. Investigate the root causes of anomalies to determine if they were triggered by system issues, attacker behavior, or external factors.
                        2. Strengthen detection mechanisms during anticipated periods of increased activity, as identified in the intrusion attempts forecast.
                        3. Conduct regular audits and testing of detection systems to ensure consistent and reliable performance.
                        4. Optimize thresholds and refine detection logic to minimize false positives and negatives.

                    # High-Level Insights:
                        1. Correlated Trends: The upward trend in intrusion attempts and the relatively stable detection rate suggest that the organization is successfully detecting threats despite the increasing attack volume.
                        2. Proactive Planning: The forecasted rise in intrusion attempts highlights the importance of scaling up security measures and optimizing response processes to handle the growing threat landscape.
                        3. Anomaly Management: Addressing the identified anomalies in the detection rate provides an opportunity to improve detection accuracy and maintain system resilience.""", source="assistant"),
                TextMessage(
                    content="""Let's start by gathering the categories and their corresponding values for your radar chart:\n\n**Categories:**\n1. Vulnerability Score\n2. Detection Rate\n3. Response Time\n4. Threat Intelligence\n5. System Uptime\n\n**Values:**\n- Vulnerability Score: 8\n- Detection Rate: 7\n- Response Time: 6\n- Threat Intelligence: 9\n- System Uptime: 8\n\nNow, I need to confirm that all values are on a scale of 0-10 and that the number of categories matches the number of values. \n\nSince you have five categories, you'll also need to ensure you have five corresponding values, which you do.\n\nI will proceed to generate the radar chart now!""", source="assistant"),
                TextMessage(
                    content="""The radar chart has been successfully created! You can find it at the following path:

                    ./figures/cybersecurity_radar_chart_20250209_230153.png

                    Feel free to reach out if you need any further assistance or analysis!""", source="assistant"),
                TextMessage(
                    content="""Compile all the gathered information, visualizations, and insights into a professional, detailed, and actionable cybersecurity report for Contoso. Include the radar chart, analysis, and recommendations. Save the report as a PDF.""", source="assistant"),
            ],
            cancellation_token=CancellationToken(),
        )
    )

# Use asyncio.run(assistant_run_stream()) when running in a script.
await write_report_stream()

### Selector Group Chat

In [ ]:
planner_instruction = """You are a planning agent.
Your job is to break down complex tasks into smaller, manageable subtasks.
Your team members are:
    cyber_collector: The cyber collector can collect information about cyber security from its knowledge base.
    report_writer: An advanced AI agent responsible for integrating information and analysis results from various agents to produce a professional, detailed, and actionable cybersecurity report.
    security_evaluator: AI agent that assists users in generating radar charts for cybersecurity metrics.
    data_analyzer: An agent that analyzes data using forecasting or anomaly detection.
    db_reader: This agent fetches data from a specified SQLite database table, saves it as a CSV file, and returns the status and file path in JSON format.
    time_keeper: An AI agent specialized in providing accurate and real-time time information.

 You only plan and delegate tasks - you do not execute them yourself.

 When assigning tasks, use this format:
 1. <agent> : <task>


## Guidelines to generate a cyber security report
1. Start with the time_keeper to get the current time.
2. Allocate the cyber_collector to retrieve latest threat information.
3. Use the metric types from the cyber_collector to have the db_reader fetch database data.
4. Use the CSV files from the db_reader to have the data analyzer perform analysis, including forecasting and anomaly detection.
5. Allocate the security_evaluator to create radar chart of evaluation metrics.
6. Start report_writer to compile information and generate PDF report.


After user's question is fully answered and all tasks are complete, summarize the findings and end with "TERMINATE".
"""


planning_agent = AssistantAgent(
    "planning_agent",
    description="An agent for planning tasks, this agent should be the first to engage when given a new task.",
    model_client=aoai_client,
    system_message=planner_instruction,
)

In [ ]:
from autogen_agentchat.conditions import MaxMessageTermination, TextMentionTermination
from autogen_agentchat.teams import SelectorGroupChat


text_mention_termination = TextMentionTermination("TERMINATE")
max_messages_termination = MaxMessageTermination(max_messages=25)
termination = text_mention_termination | max_messages_termination

team = SelectorGroupChat(
    [planning_agent, time_keeper, cyber_collector, db_reader,
        data_analyzer, security_evaluator, report_writer],
    model_client=aoai_client,
    termination_condition=termination,
)

In [ ]:
from autogen_agentchat.ui import Console


# await team.reset()  # Reset the team for the next run.

task = "Generate a cybersecurity report for Contoso."

# Use asyncio.run(...) if you are running this in a script.
await Console(team.run_stream(task=task))

## Custom Selector Function
Often times we want better control over the selection process. To this end, we can set the `selector_func` argument with a custom selector function to override the default model-based selection. For instance, we want the Planning Agent to speak immediately after any specialized agent to check the progress.

In [ ]:
from typing import Sequence
from autogen_agentchat.messages import ChatMessage


def selector_func(messages: Sequence[ChatMessage]) -> str | None:
    if messages[-1].source != planning_agent.name:
        return planning_agent.name
    return None

team2 = SelectorGroupChat(
    [planning_agent, time_keeper, cyber_collector, db_reader,
        data_analyzer, security_evaluator, report_writer],
    model_client=aoai_client,
    termination_condition=termination,
    selector_func=selector_func,
)

await Console(team2.run_stream(task=task))

In [ ]:
await team2.reset()  # Reset the team for the next run.